## Querying nasa exoplanet archive
The function 'nasa_exo' takes as input the SWEET-Cat table and adds lines to a text file for each of the stars in SWEET-Cat with 4 columns corresponding, in this order, to the value of vsini, the upper bound, the lower bound, the bibcode and the catalog name ('nasa_exoplanet_archive').

In [1]:
# imports

# SWEET-Cat
import urllib
import pandas as pd
import numpy as np
from astropy import coordinates as coord
from astropy import units as u

from astropy.table import Table
from astropy.io.ascii import convert_numpy

# astroquery
from astroquery.ipac.nexsci.nasa_exoplanet_archive import NasaExoplanetArchive as nas
from astroquery.utils.tap.core import TapPlus #tap service

In [2]:
#Download SWEET-Cat
sweetCat_table_url = "https://sweetcat.iastro.pt/catalog/SWEETCAT_Dataframe.csv"
converters={'gaia_dr2': [convert_numpy(np.int64)],'gaia_dr3': [convert_numpy(np.int64)] }
sweet_table = Table.read(sweetCat_table_url, encoding='UTF-8',format='csv', converters=converters)

sweet_stars = sweet_table['Name']

# Nasa
nasa_exo = TapPlus(url="https://exoplanetarchive.ipac.caltech.edu/TAP/")

In [10]:
sweet_table

Name,hd,RA,DEC,Vmag,eVmag,PlxFlag,Teff,eTeff,Logg,eLogg,Vt,eVt,[Fe/H],e[Fe/H],Reference,Link,SWFlag,Update,Comment,Database,gaia_dr2,gaia_dr3,Plx,ePlx,Gmag,eGmag,RPmag,eRPmag,BPmag,eBPmag,FG,eFG,G_flux_std_n,Logg_gaia,eLogg_gaia,Mass_t,eMass_t,Radius_t,eRadius_t,spec_base,Distance,RA_EU,DEC_EU,RA_NASA,DEC_NASA,Distance_b,eDistance_b
str29,str8,str11,str12,float64,str5,str8,float64,float64,float64,float64,str5,str5,float64,float64,str42,str112,int32,str10,str35,str7,int64,int64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,str40,float64,float64,float64,float64,float64,float64,float64
11 Com,107383,12 20 43.02,+17 47 34.33,4.74,0.02,GAIAeDR3,4824.0,28.0,2.53,0.09,1.48,0.03,-0.26,0.02,Sousa et al. 2021,https://ui.adsabs.harvard.edu/abs/2021arXiv210904781S/abstract,1,2021-01-01,--,"EU,NASA",3946945413106333696,3946945413106333696,10.1677,0.1365,4.4373,0.0029,3.7715,0.0046,4.9591,0.003,316300000.0,270900.0,0.0142562,2.396255611746718,0.0158830917888943,2.586983174571127,0.0220003678346113,17.168076035374188,0.5624503051374177,11Com_HARPSS_115000_378_691_2020,98.3506594411716,185.1791667,17.792777800000003,185.1787793,17.7932516,97.9758759,1.3001404000000036
11 UMi,136726,15 17 05.88,+71 49 26.04,5.01,0.01,GAIAeDR3,4314.0,93.0,1.82,0.31,1.67,0.09,-0.05,0.04,Sousa et al. 2021,https://ui.adsabs.harvard.edu/abs/2021arXiv210904781S/abstract,1,2021-01-01,--,"EU,NASA",1696798367260229376,1696798367260229376,7.926,0.0874,4.5567,0.0029,3.7446,0.0062,5.2733,0.0031,283300000.0,262200.0,0.0132515,2.0152807045039647,0.0557398887466734,3.034747445422418,0.0325959194092446,25.076026661910657,0.8215258843544753,11UMi_NARVAL_2014,126.167045167802,229.275,71.82388890000001,229.2745954,71.8239428,126.057068,1.3480990000000048
14 And,221345,23 31 17.41,+39 14 10.30,5.22,--,GAIAeDR3,4745.0,38.0,2.52,0.1,1.5,0.04,-0.28,0.03,Sousa et al. 2021,https://ui.adsabs.harvard.edu/abs/2021arXiv210904781S/abstract,1,2021-01-01,--,"EU,NASA",1920113512486282240,1920113512486282240,13.1681,0.0727,4.9222,0.0028,4.2366,0.0045,5.4557,0.0029,202300000.0,110800.0,0.0113062999999999,2.666530786926588,0.0190349773810214,2.020961547844592,0.0144748255018188,11.101348164246616,0.3636957717074285,14And_SOPHIE_HR_2020,75.94110008277579,352.8208333,39.2361111,352.82415,39.2358367,75.8471832,0.4380110999999971
14 Her,145675,16 10 24.31,+43 49 03.52,6.67,--,GAIAeDR3,5360.0,57.0,4.16,0.14,0.94,0.09,0.37,0.04,Sousa et al. 2021,https://ui.adsabs.harvard.edu/abs/2021arXiv210904781S/abstract,1,2021-01-01,--,"EU,NASA",1385293808145621504,1385293808145621504,55.8657,0.0291,6.3952,0.0028,5.8111,0.0038,6.8131,0.0029,52100000.0,9048.0,0.0030675,4.445040579625312,0.0267588009085265,0.939957930643933,0.0141325926888806,1.0216696216954395,0.0334713330304576,14Her_SOPHIE_HR_2020,17.900071063282102,242.5958333,43.8216667,242.6021014,43.816362100000006,17.8923416,0.0102347999999992
16 Cyg B,186427,19 41 51.97,+50 31 03.08,6.2,--,GAIAeDR3,5785.0,23.0,4.37,0.04,1.04,0.03,0.09,0.02,Sousa et al. 2021,https://ui.adsabs.harvard.edu/abs/2021arXiv210904781S/abstract,1,2021-01-01,--,"EU,NASA",2135550755683407232,2135550755683407232,47.3302,0.0171,6.0734,0.0028,5.5746,0.0039,6.4007,0.0029,70080000.0,17930.0,0.0041092,4.358204493747518,0.0096069708081801,1.0223399401784177,0.0067772304252642,1.1509191640247949,0.0377057297311846,16CygB_HARPSN_2020,21.1281591880026,295.4625,50.5175,295.4656422,50.5168243,21.1187477,0.0084534000000005
18 Del,199665,20 58 25.93,+10 50 21.42,5.51,0.01,GAIAeDR3,5089.0,34.0,3.07,0.1,1.37,0.03,0.03,0.03,Sousa et al. 2021,https://ui.adsabs.harvard.edu/abs/2021arXiv210904781S/abstract,1,2021-01-01,--,"EU,NASA",1756741374681702784,1756741374681702784,13.3021,0.0652,5.2693,0.0028,4.6413,0.0041,5.7357,0.0028,147000000.0,61180.0,0.0098169,2.957759500571873,0.0148487989177627,1.9470225357239563,0.0130257343860771,7.670333011243995,0.2512908921064065,18Del_HARPSS_115000_378_691_2020,75.1761000142835,

In [8]:
arch = nas.query_criteria(table="ps") #takes a while to run

In [9]:
def nasa_exo(sweet_table):
    
    sweet_stars = sweet_table['Name']
    
    gaia = sweet_table['gaia_dr2']
    
    identifiers = []
    for i in gaia:
        identifiers.append("Gaia DR2 " + str(i))
    
    #arch = nas.query_criteria(table="ps")
    
    result_nasa = arch[np.in1d(arch['gaia_id'], identifiers)]
    
    result_nasa = result_nasa["hostname","st_vsin","st_vsinerr1","st_vsinerr2","pl_refname"]
    
    df_nasa = result_nasa.to_pandas()
    df_nasa = df_nasa.dropna(subset=['st_vsin'])
    df_nasa = df_nasa.drop_duplicates()
    
    #total dataframe
    #df_nasa = df_nasa.drop(['hostname'], axis=1)
    cat = ['nasa_exoplanet_archive']*len(df_nasa)
    df_nasa['Catalogue'] = cat
    
    #dataframe for each star
    for star_name in sweet_stars:
        gaia2_id = sweet_table[sweet_table['Name'] == star_name]['gaia_dr2']
        star_id2 = "Gaia DR2 " + str(gaia2_id[0])
        df_star = df_nasa[df_nasa['hostname'] == star_name]
    
        # Make csv file with the values
        star_id2_simbad = "gaia_dr2 " + str(gaia2_id[0])
        if "/" in star_name or "*" in star_name:
            file_name = star_id2_simbad
        else:
            file_name = star_name
            
        df_star = df_star.drop(['hostname'], axis=1)
        df_star.to_csv('{}.txt'.format(file_name), mode='a', index=False, header=False, sep = '\t')
    
    return 'nasa done'

In [11]:
nasa_exo(sweet_table)

'nasa done'

## adapting for one star

In [10]:
sweet_stars = sweet_table['Name']
    
gaia = sweet_table['gaia_dr2']
    
identifiers = []
for i in gaia:
    identifiers.append("Gaia DR2 " + str(i))
    
#arch = nas.query_criteria(table="ps")
    
result_nasa = arch[np.in1d(arch['gaia_id'], identifiers)]
    
result_nasa = result_nasa["gaia_id","st_vsin","st_vsinerr1","st_vsinerr2","st_refname"]
    
df_nasa = result_nasa.to_pandas()
df_nasa = df_nasa.dropna(subset=['st_vsin'])
df_nasa = df_nasa.drop_duplicates()

def nasa_exo(star_name, sweet_table, df_nasa):
    
    gaia2_id = sweet_table[sweet_table['Name'] == star_name]['gaia_dr2']
    star_id2 = "Gaia DR2 " + str(gaia2_id[0])
    
    #dataframe for each star
    df_star = df_nasa[df_nasa["gaia_id"] == star_id2]
    df_star = df_star.drop(['gaia_id'], axis=1)
    cat = ['nasa_exoplanet_archive']*len(df_star)
    df_star['Catalogue'] = cat
    
    # Make csv file with the values
    star_id2_simbad = "gaia_dr2 " + str(gaia2_id[0])
    if "/" in star_name or "*" in star_name:
        file_name = star_id2_simbad
    else:
        file_name = star_name

    df_star.to_csv('{}.txt'.format(file_name), mode='a', index=False, header=False, sep = '\t')
    
    return 

In [13]:
nasa_exo('11 Com', sweet_table, df_nasa)

## Now we're gonna get just the results with the default sets (flag = 1), in 2 ways

### Method 1: Querying the catalog

In [12]:
# Pedro's notebook
sweetCat_table_url = "http://sweetcat.iastro.pt/catalog/SWEETCAT_Dataframe.csv"
dtype_SW = dtype={'gaia_dr2':'int64','gaia_dr3':'int64'}
SC = pd.read_csv(urllib.request.urlopen(sweetCat_table_url), dtype=dtype_SW)

Gaia2=list(SC['gaia_dr2'])
Gaia2_id=[]

for i in Gaia2:
    Gaia2_id.append("Gaia DR2 " + str(i))

#identifiers=list(SC_name)

from astroquery.ipac.nexsci.nasa_exoplanet_archive import NasaExoplanetArchive as nas

AA = nas.query_criteria(table="ps")
if False:
    AA=AA.to_pandas()
stars_def1=AA[AA["default_flag"]==1]
new_Stars_SW=stars_def1[np.in1d(stars_def1["gaia_id"], Gaia2_id)]
catalogo=new_Stars_SW[["hostname","st_vsin","st_vsinerr1","st_vsinerr2","pl_refname"]]
catalogo_final=catalogo.to_pandas().drop_duplicates()

if False:
    catalogo_final.to_csv("Catalogo_nasa.csv", index=False)
catalogo_final

,hostname,st_vsin,st_vsinerr1,st_vsinerr2,pl_refname
0,OGLE-TR-10,NaN,NaN,NaN,<a refstr=TORRES_ET_AL__2008 href=https://ui.a...
1,BD-08 2823,1.40,NaN,NaN,<a refstr=HEBRARD_ET_AL__2010 href=https://ui....
2,HD 110014,2.00,NaN,NaN,<a refstr=DE_MEDEIROS_ET_AL__2009 href=https:/...
3,HIP 5158,NaN,NaN,NaN,<a refstr=LO_CURTO_ET_AL__2010 href=https://ui...
4,HD 44219,2.22,1.00,-1.0,<a refstr=NAEF_ET_AL__2010 href=https://ui.ads...
...,...,...,...,...,...
4821,GJ 3470,NaN,NaN,NaN,<a refstr=AWIPHAN_ET_AL__2016 href=https://ui....
4823,OGLE2-TR-L9,39.33,0.38,NaN,<a refstr=SNELLEN_ET_AL__2009 href=https://ui....
4825,EPIC 246851721,74.92,0.62,-0.6,<a refstr=YU_ET_AL__2018 href=https://ui.adsab...
4829,HD 149026,NaN,NaN,NaN,<a refstr=STASSUN_ET_AL__2017 href=https://ui....


###  Method 2: From the table downloaded from the website, with the Default already set to 1

In [3]:
table = pd.read_csv('PS_2023.03.17_05.36.42.csv', header=0)
table.pop('pl_name')
table

,hostname,gaia_id,default_flag,st_refname,st_vsin,st_vsinerr1,st_vsinerr2,st_vsinlim,rowupdate,releasedate;;
0,11 Com,Gaia DR2 3946945413106333696,1,<a refstr=LIU_ET_AL__2008 href=https://ui.adsa...,1.20,1.0,-1.0,0.0,2014-05-14,2014-05-14;;
1,11 UMi,Gaia DR2 1696798367260229376,1,<a refstr=STASSUN_ET_AL__2017 href=https://ui....,NaN,NaN,NaN,NaN,2018-09-04,2018-09-06;;
2,14 And,Gaia DR2 1920113512486282240,1,<a refstr=SATO_ET_AL__2008 href=https://ui.ads...,2.60,NaN,NaN,0.0,2014-05-14,2014-05-14;;
3,14 Her,Gaia DR2 1385293808145621504,1,<a refstr=FENG_ET_AL__2022 href=https://ui.ads...,NaN,NaN,NaN,NaN,2022-09-19,2022-09-19;;
4,16 Cyg B,Gaia DR2 2135550755683407232,1,<a refstr=STASSUN_ET_AL__2017 href=https://ui....,NaN,NaN,NaN,NaN,2018-09-04,2018-09-06;;
...,...,...,...,...,...,...,...,...,...,...
5295,ups And,Gaia DR2 348020448377061376,1,<a refstr=CURIEL_ET_AL__2011 href=https://ui.a...,NaN,NaN,NaN,NaN,2019-01-28,2019-01-31;;
5296,ups And,Gaia DR2 348020448377061376,1,<a refstr=CURIEL_ET_AL__2011 href=https://ui.a...,NaN,NaN,NaN,NaN,2019-01-28,2019-01-31;;
5297,ups And,Gaia DR2 348020448377061376,1,<a refstr=CURIEL_ET_AL__2011 href=https://ui.a...,NaN,NaN,NaN,NaN,2019-01-28,2019-01-31;;
5298,ups Leo,Gaia DR2 3794167001116433152,1,<a refstr=TENG_ET_AL__2021 href=https://ui.ads...,1.78,NaN,NaN,0.0,2022-01-10,2022-01-10;;


In [4]:
# Filtering the table just for SWEET-Cat stars
sweet_stars = sweet_table['Name']
    
gaia = sweet_table['gaia_dr2']
    
identifiers = []
for i in gaia:
    identifiers.append("Gaia DR2 " + str(i))
    
for i, star in enumerate(table['gaia_id']):
    if pd.isna(star):
        table.drop(labels=i, axis=0,inplace=True)
        continue
    if star not in identifiers:
        #print(star)
        table.drop(labels=i, axis=0, inplace=True)

print(table)
print(len(table))

      hostname                       gaia_id  default_flag  \
0       11 Com  Gaia DR2 3946945413106333696             1   
1       11 UMi  Gaia DR2 1696798367260229376             1   
2       14 And  Gaia DR2 1920113512486282240             1   
3       14 Her  Gaia DR2 1385293808145621504             1   
4     16 Cyg B  Gaia DR2 2135550755683407232             1   
...        ...                           ...           ...   
5295   ups And   Gaia DR2 348020448377061376             1   
5296   ups And   Gaia DR2 348020448377061376             1   
5297   ups And   Gaia DR2 348020448377061376             1   
5298   ups Leo  Gaia DR2 3794167001116433152             1   
5299    xi Aql  Gaia DR2 4298361114750843904             1   

                                             st_refname  st_vsin  st_vsinerr1  \
0     <a refstr=LIU_ET_AL__2008 href=https://ui.adsa...     1.20          1.0   
1     <a refstr=STASSUN_ET_AL__2017 href=https://ui....      NaN          NaN   
2     <a ref

In [6]:
table.to_csv('_nasa_table.txt', index=False, sep = '\t')

In [11]:
# Writing a function that takes as an argument the name of the SWEET-Cat star and returns the vsini with the respective errors and reference

def nasa_vsini(star, table):
    gaia2_id = sweet_table[sweet_table['Name'] == star]['gaia_dr2']
    star_id2 = "Gaia DR2 " + str(gaia2_id[0])
    return table[table['gaia_id']==star_id2]

In [16]:
nasa_vsini('55 Cnc', table)

,hostname,gaia_id,default_flag,st_refname,st_vsin,st_vsinerr1,st_vsinerr2,st_vsinlim,rowupdate,releasedate;;
27,55 Cnc,Gaia DR2 704967037090946688,1,<a refstr=BOURRIER_ET_AL__2018 href=https://ui...,1.23,0.01,-0.01,1.0,2021-02-05,2021-02-11;;
28,55 Cnc,Gaia DR2 704967037090946688,1,<a refstr=BOURRIER_ET_AL__2018 href=https://ui...,1.23,0.01,-0.01,1.0,2021-02-05,2021-02-11;;
29,55 Cnc,Gaia DR2 704967037090946688,1,<a refstr=BOURRIER_ET_AL__2018 href=https://ui...,1.23,0.01,-0.01,1.0,2021-02-05,2021-02-11;;
30,55 Cnc,Gaia DR2 704967037090946688,1,<a refstr=BOURRIER_ET_AL__2018 href=https://ui...,1.23,0.01,-0.01,1.0,2021-02-05,2021-02-11;;
31,55 Cnc,Gaia DR2 704967037090946688,1,<a refstr=BOURRIER_ET_AL__2018 href=https://ui...,1.23,0.01,-0.01,1.0,2021-02-05,2021-02-11;;
